# gathering thingiverse things

an effective scraping and caching strategy for gathering information on thingiverse.

each activity requires:
1. making requests
2. tidying requests
3. creating tasks to save salient data, create a file witht he missing columsn


1. gather the index of things from search
2. save the index to disk as parquet
3. read the index of things from disks
4. save the things to disc

the second request is solely to get the description information about the thing cause that is not included in the search response
despite the openapi schema saying that it should be


In [1]:
    __import__("dotenv").load_dotenv()
    params = dict(access_token=os.environ["THINGIVERSE_ACCESS_TOKEN"])
    import requests_cache
    cache = platformdirs.user_cache_path("a11yhood") / "thingiverse"
    search_session = requests_cache.CachedSession(cache / "search_responses.sqlite")
    thing_session = requests_cache.CachedSession(cache / "thing_responses.sqlite")

In [2]:
    def search_tags(term="assistive+technology", pages=None, per_page=30):
        """search thing"""
        page = 1
        while not pages or page <= pages:
            response = search_session.get(f"https://api.thingiverse.com/search/{term}", params | dict(per_page=per_page, page=page, type="thing"),)
            if pages is None:
                pages = (response.json()["total"] // per_page) + 1
            page += 1

In [7]:
    search_tags()

In [8]:
    results = (
        Series(search_session.cache.urls())
        .apply(search_session.get)
        .apply(operator.methodcaller("json"))
        .apply(lambda x: x.get("hits", []))
        .explode()
        .apply(Series)
        .set_index("public_url")
    )

In [9]:
    for j, (i, row) in enumerate(results.dropna(subset="id").iterrows()):
        time.sleep(random.random()*2)
        thing_session.get(row.url, params=params)
        print(i, j)

https://www.thingiverse.com/thing:1852950 0
https://www.thingiverse.com/thing:1871831 1
https://www.thingiverse.com/thing:1103082 2
https://www.thingiverse.com/thing:2745606 3
https://www.thingiverse.com/thing:3057298 4
https://www.thingiverse.com/thing:5936465 5
https://www.thingiverse.com/thing:2785602 6
https://www.thingiverse.com/thing:2801872 7
https://www.thingiverse.com/thing:230209 8
https://www.thingiverse.com/thing:1105307 9
https://www.thingiverse.com/thing:3929118 10
https://www.thingiverse.com/thing:6208844 11
https://www.thingiverse.com/thing:1104871 12
https://www.thingiverse.com/thing:3929193 13
https://www.thingiverse.com/thing:3086651 14
https://www.thingiverse.com/thing:6303151 15
https://www.thingiverse.com/thing:302309 16
https://www.thingiverse.com/thing:3929230 17
https://www.thingiverse.com/thing:3929439 18
https://www.thingiverse.com/thing:3959612 19
https://www.thingiverse.com/thing:6801694 20
https://www.thingiverse.com/thing:3086688 21
https://www.thingivers

In [388]:
    def get_thing_details_tidy(df, index):
        df = df[df.columns.difference(index.columns)]
        df = df.join(
            df.pop("zip_data").series().stack().explode().apply(operator.itemgetter("url")).groupby(level=0).agg(list).rename("files")
        )
        # the default image is in the files
        df.pop("default_image");
        df.pop("edu_details_parts");
        df.pop("education");
        df.pop("description_html");
        # TODO there is information in details_parts than can be extracted for search.
        details_parts = df.pop("details_parts").explode().apply(Series).set_index("type", append=True)
        # detail_parts is segmented string info we cram into  a single block
        details_parts = details_parts.join(
            details_parts.data.explode().dropna().apply(Series).fillna("").apply("\n".join, axis=1).rename("parts")
        )
        details_parts = details_parts.reset_index(1, drop=True)
        df = df.join(
            (("<h2>" + details_parts["name"] + "</h2>") + "\n" + details_parts["parts"]).dropna().groupby(level=0).agg("\n".join).rename("details_parts")
        )
        df["ancestors"] = df.ancestors.explode().dropna().series().public_url.groupby(level=0).agg(list)
        return df

we have to make two layers of requests.

1. the search request
2. the things requests

at each stage we should saving the bulk to be efficient with effort

In [8]:
    things = get_tidy_responses(*thingiverse.search_tags(pages=2))

we are not search for authors, but that information should be added later

* categorical data is best represented as series
* text and numerical features are best represented

In [17]:
things

,name,url,public_url,created_at,thumbnail,preview_image,creator,is_private,is_purchased,is_published,...,download_count,view_count,education,remix_count,app_count,root_comment_count,is_derivative,ancestors,type_name,zip_data
id,,,,,,,,,,,,,,,,,,,,,
1852950,Open Assistive Technology - Key Turner,https://api.thingiverse.com/things/1852950,https://www.thingiverse.com/thing:1852950,2016-10-27T01:46:07+00:00,https://cdn.thingiverse.com/renders/11/e3/d6/0...,https://cdn.thingiverse.com/renders/11/e3/d6/0...,"{'id': 957394, 'name': 'ivan_gee', 'first_name...",0,0,1,...,4049,14974,"{'grades': [], 'subjects': []}",2,4,5,False,[],Thing,{'files': [{'name': 'Assistive_Key_Turner_3A.S...
1871831,Open Assistive Technology - Bag Holder,https://api.thingiverse.com/things/1871831,https://www.thingiverse.com/thing:1871831,2016-11-05T01:50:01+00:00,https://cdn.thingiverse.com/renders/2a/55/85/8...,https://cdn.thingiverse.com/renders/2a/55/85/8...,"{'id': 957394, 'name': 'ivan_gee', 'first_name...",0,0,1,...,1578,8982,"{'grades': [], 'subjects': []}",4,4,1,False,[],Thing,{'files': [{'name': 'Assistive_Bag_Holder_Part...
1103082,Easy Zipper for Assistive Technology Challenge,https://api.thingiverse.com/things/1103082,https://www.thingiverse.com/thing:1103082,2015-10-31T12:20:01+00:00,https://cdn.thingiverse.com/renders/ef/fa/1a/f...,https://cdn.thingiverse.com/renders/ef/fa/1a/f...,"{'id': 342259, 'name': 'miginv', 'first_name':...",0,0,1,...,2896,9170,"{'grades': [], 'subjects': []}",4,4,5,False,[],Thing,"{'files': [{'name': 'easyZypper7.stl', 'url': ..."
2745606,Assistive Technology Mouse (ATMouse),https://api.thingiverse.com/things/2745606,https://www.thingiverse.com/thing:2745606,2018-01-04T07:53:20+00:00,https://cdn.thingiverse.com/renders/1c/31/d9/b...,https://cdn.thingiverse.com/renders/1c/31/d9/b...,"{'id': 650208, 'name': 'lowendguru', 'first_na...",0,0,1,...,1493,1691,"{'grades': [], 'subjects': []}",0,4,1,False,[],Thing,"{'files': [{'name': 'front_v4.stl', 'url': 'ht..."
3057298,Assistive Technology - Bag Holder,https://api.thingiverse.com/things/3057298,https://www.thingiverse.com/thing:3057298,2018-08-20T13:49:03+00:00,https://cdn.thingiverse.com/renders/21/c1/62/1...,https://cdn.thingiverse.com/renders/21/c1/62/1...,"{'id': 2210107, 'name': 'Head2Hand', 'first_na...",0,0,1,...,584,1827,"{'grades': [], 'subjects': []}",1,4,2,True,"[{'id': 1871831, 'name': 'Open Assistive Techn...",Thing,"{'files': [{'name': 'Middle_Part.stl', 'url': ..."


In [16]:
%% --md
```mermaid
graph TD;
    source --->|scrape| data;
    data --->|feature engineer| dataframes

```


```mermaid
graph TD;
    source --->|scrape| data;
    data --->|feature engineer| dataframes
```

In [ ]:
class Source:
    async def get_data_text(self):
        pass

    async def get_data_tags(self):
        pass

In [32]:
things.head(2).T.style

id,1852950,1871831
name,Open Assistive Technology - Key Turner,Open Assistive Technology - Bag Holder
url,https://api.thingiverse.com/things/1852950,https://api.thingiverse.com/things/1871831
public_url,https://www.thingiverse.com/thing:1852950,https://www.thingiverse.com/thing:1871831
created_at,2016-10-27T01:46:07+00:00,2016-11-05T01:50:01+00:00
thumbnail,https://cdn.thingiverse.com/renders/11/e3/d6/0e/af/7499a32917cd2f6b8e4d359a1cb0ecd3_thumb_medium.jpg,https://cdn.thingiverse.com/renders/2a/55/85/86/01/1a19a70783ce94d22c9a9f3157ebc453_thumb_medium.jpg
preview_image,https://cdn.thingiverse.com/renders/11/e3/d6/0e/af/7499a32917cd2f6b8e4d359a1cb0ecd3_preview_card.jpg,https://cdn.thingiverse.com/renders/2a/55/85/86/01/1a19a70783ce94d22c9a9f3157ebc453_preview_card.jpg
creator,"{'id': 957394, 'name': 'ivan_gee', 'first_name': 'Ivan', 'last_name': 'Gee', 'url': 'https://api.thingiverse.com/users/ivan_gee', 'public_url': 'https://www.thingiverse.com/ivan_gee', 'thumbnail': 'https://cdn.thingiverse.com/site/img/default/avatar/avatar_designer_thumb_medium.jpg', 'count_of_followers': 17, 'count_of_following': 4, 'count_of_designs': 4, 'collection_count': 4, 'make_count': 2, 'accepts_tips': False, 'is_following': False, 'location': 'Vancouver BC CANADA', 'cover': 'https://cdn.thingiverse.com/site/img/default/cover/cover-3_preview_large.jpg', 'is_admin': False, 'is_moderator': False, 'is_featured': False, 'is_verified': False}","{'id': 957394, 'name': 'ivan_gee', 'first_name': 'Ivan', 'last_name': 'Gee', 'url': 'https://api.thingiverse.com/users/ivan_gee', 'public_url': 'https://www.thingiverse.com/ivan_gee', 'thumbnail': 'https://cdn.thingiverse.com/site/img/default/avatar/avatar_designer_thumb_medium.jpg', 'count_of_followers': 17, 'count_of_following': 4, 'count_of_designs': 4, 'collection_count': 4, 'make_count': 2, 'accepts_tips': False, 'is_following': False, 'location': 'Vancouver BC CANADA', 'cover': 'https://cdn.thingiverse.com/site/img/default/cover/cover-3_preview_large.jpg', 'is_admin': False, 'is_moderator': False, 'is_featured': False, 'is_verified': False}"
is_private,0,0
is_purchased,0,0
is_published,1,1


In [25]:
things.tags

id
1852950    [3dprintable, 3dprinting, accessibility, assis...
1871831    [3dprintable, 3dprinting, accessibility, assis...
1103082    [assistive, assistivetech, assistive_technolog...
2745606    [assistive, assistivetech, assistive_technolog...
3057298    [als, assistivetech, assistive_device, assisti...
Name: tags, dtype: object

In [46]:
thingsthings.dtypes.to_frame().style

,0
id,int64
name,object
thumbnail,object
url,object
public_url,object
creator,object
added,object
modified,object
is_published,int64
is_wip,int64
